In [3]:
# Step 1: Upload your kaggle.json file (API token)
from google.colab import files
files.upload()

# Step 2: Create a Kaggle directory and move the kaggle.json file to it
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Step 3: Set permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json

# Step 4: Download the retinal disease classification dataset using the Kaggle API
!kaggle datasets download -d kmader/parkinsons-drawings

# Step 5: Unzip the downloaded dataset
!unzip parkinsons-drawings.zip -d ./parkinsons-drawings/


Saving kaggle.json to kaggle (2).json
Dataset URL: https://www.kaggle.com/datasets/kmader/parkinsons-drawings
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
parkinsons-drawings.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  parkinsons-drawings.zip
  inflating: ./parkinsons-drawings/drawings/spiral/testing/healthy/V01HE01.png  
  inflating: ./parkinsons-drawings/drawings/spiral/testing/healthy/V02HE01.png  
  inflating: ./parkinsons-drawings/drawings/spiral/testing/healthy/V03HE1.png  
  inflating: ./parkinsons-drawings/drawings/spiral/testing/healthy/V04HE01.png  
  inflating: ./parkinsons-drawings/drawings/spiral/testing/healthy/V05HE01.png  
  inflating: ./parkinsons-drawings/drawings/spiral/testing/healthy/V06HE01.png  
  inflating: ./parkinsons-drawings/drawings/spiral/testing/healthy/V07HE01.png  
  inflating: ./parkinsons-drawings/drawings/spiral/testing/healthy/V08HE01.png  
  inflating: .

In [5]:
# Step 6: Display the path of the dataset
import os
dataset_path = os.path.abspath("./parkinsons-drawings/")
print(f"Dataset extracted to: {dataset_path}")


Dataset extracted to: /content/parkinsons-drawings


In [7]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

In [10]:
data_dir = 'parkinsons-drawings/drawings'

# Directory paths for spiral and wave drawings
spiral_train_dir = os.path.join(data_dir, 'spiral/training')
spiral_test_dir = os.path.join(data_dir, 'spiral/testing')

wave_train_dir = os.path.join(data_dir, 'wave/training')
wave_test_dir = os.path.join(data_dir, 'wave/testing')

Data Augmentation and Generators


In [11]:
datagen = ImageDataGenerator(rescale=1./255)

# Spiral training and testing datasets
spiral_train_gen = datagen.flow_from_directory(spiral_train_dir, target_size=(128, 128),
                                               class_mode='binary', batch_size=32)
spiral_test_gen = datagen.flow_from_directory(spiral_test_dir, target_size=(128, 128),
                                              class_mode='binary', batch_size=32)

# Wave training and testing datasets
wave_train_gen = datagen.flow_from_directory(wave_train_dir, target_size=(128, 128),
                                             class_mode='binary', batch_size=32)
wave_test_gen = datagen.flow_from_directory(wave_test_dir, target_size=(128, 128),
                                            class_mode='binary', batch_size=32)


Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.
Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


Model Creation (for Spiral)

In [16]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on spiral data
history = model.fit(spiral_train_gen, epochs=100, validation_data=spiral_test_gen)

# Save the model for future use (e.g., connecting with Django)
model.save('parkinsons_spiral_model.h5')


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 754ms/step - accuracy: 0.5000 - loss: 1.1093 - val_accuracy: 0.5000 - val_loss: 0.7095
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 594ms/step - accuracy: 0.5359 - loss: 0.7015 - val_accuracy: 0.5000 - val_loss: 0.7142
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 835ms/step - accuracy: 0.5625 - loss: 0.7276 - val_accuracy: 0.5000 - val_loss: 0.6978
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 858ms/step - accuracy: 0.5104 - loss: 0.6862 - val_accuracy: 0.5000 - val_loss: 0.6938
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.5000 - loss: 0.6917 - val_accuracy: 0.5000 - val_loss: 0.6907
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 599ms/step - accuracy: 0.5295 - loss: 0.6901 - val_accuracy: 0.5000 - val_loss: 0.6894
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 595ms/step - accuracy: 0.5000 - loss: 0.6845 - val_accuracy: 0.5000 - val_loss: 0.6851
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 608ms/step - accuracy: 0.4036 - loss: 0.6866 - val_accuracy: 0.5000 - val_

In [17]:
# Train the model on spiral data
history = model.fit(spiral_train_gen, epochs=100, validation_data=spiral_test_gen)

# Save the model for future use (e.g., connecting with Django)
model.save('parkinsons_spiral_model.h5')


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 641ms/step - accuracy: 1.0000 - loss: 1.5030e-04 - val_accuracy: 0.7667 - val_loss: 1.0685
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 602ms/step - accuracy: 1.0000 - loss: 1.8313e-04 - val_accuracy: 0.7667 - val_loss: 1.0646
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 1.0000 - loss: 1.1709e-04 - val_accuracy: 0.7667 - val_loss: 1.0582
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 618ms/step - accuracy: 1.0000 - loss: 1.3810e-04 - val_accuracy: 0.7667 - val_loss: 1.0584
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 590ms/step - accuracy: 1.0000 - loss: 1.2461e-04 - val_accuracy: 0.7667 - val_loss: 1.0622
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 854ms/step - accuracy: 1.0000 - loss: 9.8297e-05 - val_accuracy: 0.7667 - val_loss: 1.0712
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 591ms/step - accuracy: 1.0000 - loss: 1.2030e-04 - val_accuracy: 0.7667 - val_loss: 1.0787
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 1.0000 - loss: 1.2092e-04 

In [18]:
# Train the model on wave data
history_wave = model.fit(wave_train_gen, epochs=100, validation_data=wave_test_gen)

# Save the model
model.save('parkinsons_wave_model.h5')


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 977ms/step - accuracy: 0.4549 - loss: 3.0328 - val_accuracy: 0.5333 - val_loss: 0.7422
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 601ms/step - accuracy: 0.5413 - loss: 0.6854 - val_accuracy: 0.5000 - val_loss: 0.6937
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 879ms/step - accuracy: 0.3323 - loss: 0.7008 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 624ms/step - accuracy: 0.5156 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 873ms/step - accuracy: 0.4938 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 862ms/step - accuracy: 0.5000 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.5076 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 706ms/step - accuracy: 0.4983 - loss: 0.6928 - val_accuracy: 0.5333 - val_